# IC170922A_wGFU

.. contents:: :local:

## Introduction

The IC170922A_wGFU analysis is a multi-dataset time-integrated single source
analysis with a two-component likelihood function using a spacial and an energy
event PDF.

## Setup

Import analysis creation function from "i3skyllh" project.

In [1]:
from i3skyllh.analyses.IC170922A_wGFU.analysis import create_analysis

Add the required imports for the analysis.

In [2]:
import logging
import numpy as np

# Classes to define the source hypothesis.
from skyllh.physics.source import PointLikeSource

# Classes for utility functionality.
from skyllh.core.config import CFG
from skyllh.core.random import RandomStateService
from skyllh.core.timing import TimeLord

# Logging setup utilities.
from skyllh.core.debugging import (
    setup_logger,
    setup_console_handler,
    setup_file_handler
)

Define TXS 0506+056 location to use as a point source.

In [3]:
def TXS_location():
    src_ra  = np.radians(77.358)
    src_dec = np.radians(5.693)
    return (src_ra, src_dec)

Setup logging first to get log messages from "skyllh" project.

In [4]:
setup_logger('skyllh', logging.DEBUG)
log_format = '%(asctime)s %(processName)s %(name)s %(levelname)s: '\
             '%(message)s'
setup_console_handler('skyllh', logging.INFO, log_format)
#setup_file_handler('skyllh', logging.DEBUG, log_format, 'debug.log')

Set a number of CPUs available for parallel calculations.

In [5]:
CFG['multiproc']['ncpu'] = 16

Define which datasets seasons to use.

In [6]:
sample_seasons = [
    ("PointSourceTracks", "IC40"),
    ("PointSourceTracks", "IC59"),
    ("PointSourceTracks", "IC79"),
    ("PointSourceTracks", "IC86, 2011"),
    ("PointSourceTracks", "IC86, 2012"),
    ("PointSourceTracks", "IC86, 2013"),
    ("PointSourceTracks", "IC86, 2014"),
    ("PointSourceTracks", "IC86, 2015"),
    ("PointSourceTracks", "IC86, 2016"),
    ("PointSourceTracks", "IC86, 2017"),
    ("GFU", "IC86, 2015-2018")
]

Set a rss seed and define a random state service.

In [7]:
rss_seed = 1
rss = RandomStateService(rss_seed)

Define the source.

In [8]:
source = PointLikeSource(*TXS_location())

Create a TimeLord instance. It is used to keep track of execution times of specific code segments.

In [9]:
tl = TimeLord()

Create analysis.

In [10]:
with tl.task_timer('Creating analysis.'):
    ana = create_analysis(
        sample_seasons, source,
        data_base_path=None,
        compress_data=False,
        tl=tl)

[==========================================================] 100% ELT 0h:00m:01s
[==========================================================] 100% ELT 0h:00m:00s
[==========================================================] 100% ELT 0h:00m:02s
[==========================================================] 100% ELT 0h:00m:00s
[==========================================================] 100% ELT 0h:00m:03s
[==========================================================] 100% ELT 0h:00m:01s
[==========================================================] 100% ELT 0h:00m:02s
[==========================================================] 100% ELT 0h:00m:01s
[==========================================================] 100% ELT 0h:00m:17s
[==========================================================] 100% ELT 0h:00m:01s
[==========================================================] 100% ELT 0h:00m:18s
[==========================================================] 100% ELT 0h:00m:02s
[===========================

Unblind the data and print best fit parameters.

In [11]:
with tl.task_timer('Unblinding data.'):
    (TS, fitparam_dict, status) = ana.unblind(rss)
print('TS = %g'%(TS))
print('ns_fit = %g'%(fitparam_dict['ns']))
print('gamma_fit = %g'%(fitparam_dict['gamma']))

TS = 25.1273
ns_fit = 15.8888
gamma_fit = 1.98958


Generate some signal events.

In [12]:
with tl.task_timer('Generating signal events.'):
    (n_sig, signal_events_dict) = ana.sig_generator.generate_signal_events(rss, 100)
print('n_sig: %d', n_sig)
print('signal datasets: '+str(signal_events_dict.keys()))

('n_sig: %d', 98)
signal datasets: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


Print the TimeLord instance to get a summary of execution times.

In [13]:
print(tl)

Executed tasks:
[                                     Loading grl data from disk.] 0.161825 sec
[                                     Loading exp data from disk.] 2.12667 sec
[                                      Loading mc data from disk.] 52.1758 sec
[Selected only the experimental data that matches the GRL for dat] 0.0110879 sec
[                Preparing data of dataset "IC40" by "data_prep".] 0.0808389 sec
[             Appending IceCube-specific data fields to exp data.] 0.0462375 sec
[              Appending IceCube-specific data fields to MC data.] 4.34033 sec
[                                              Cleaning exp data.] 0.00128031 sec
[                                               Cleaning MC data.] 0.205514 sec
[                                          Asserting data format.] 0.000334263 sec
[Selected only the experimental data that matches the GRL for dat] 0.033159 sec
[                Preparing data of dataset "IC59" by "data_prep".] 0.276066 sec
[Selected only the 